In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function, division
import os
import time
import copy
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils, models, datasets
import torch.nn as nn
import torch.optim as optim
import nibabel as nib
import scipy.ndimage as ndi
from pathlib import Path
from PIL import Image
import io

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.13.1+cu116
Torchvision Version:  0.14.1+cu116


In [4]:
imagesFolder = '/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-240'
trainDatasetCSV = imagesFolder + '/MUESTRA_train.csv'
valDatasetCSV =   imagesFolder + '/MUESTRA_val.csv'
experimentName = 'Muestra240_1_Full_noCut'
experimentOutputFolder = '/content/gdrive/MyDrive/Tesis/Experimentos/muestra240_1'

In [5]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 10

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

usePretrained = True

learningRate = 0.0001

crossEntrophyWeigths = torch.tensor([1.0,1.0,1.0])

slicesToCut = 0

In [6]:
f = open(os.path.join(experimentOutputFolder, experimentName + "_params.txt"), "w")
f.write("batch_size: " + str(batch_size) + "\n")
f.write("epochs: " + str(num_epochs) + "\n")
f.write("feature_extract: " + str(feature_extract) + "\n")
f.write("usePretrained: " + str(usePretrained) + "\n")
f.write("learningRate: " + str(learningRate) + "\n")
f.write("cross entrophy weights: " + str(crossEntrophyWeigths) + "\n")
f.write("slicesToCut: " + str(slicesToCut) + "\n")
f.close()

In [7]:
# https://stackoverflow.com/questions/8598673/how-to-save-a-pylab-figure-into-in-memory-file-which-can-be-read-into-pil-image
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    buf = io.BytesIO()
    fig.savefig(buf, facecolor='black', dpi = 64, transparent=False) # dpi Requerido para que la imagen sea 512x512
    buf.seek(0)
    img = Image.open(buf)
    return img

In [8]:
# Modificado respecto a Muestra120_2.0
def transformGridImage(sample):
    brain_vol_data = sample.get_fdata()
    fig_rows = 4
    fig_cols = 4
    n_subplots = fig_rows * fig_cols
    n_slice = brain_vol_data.shape[2]

    slices_to_eliminate = slicesToCut

    n_slice_padding = slices_to_eliminate // 2 # quitamos los primeros y ultimos 16 slices
    n_slice = n_slice - slices_to_eliminate

    step_size = n_slice // n_subplots
    plot_range = n_subplots * step_size
    start_stop = int(n_slice_padding + (n_slice - plot_range) / 2)

    slice_indices = np.arange(n_slice_padding, n_slice_padding + n_slice, step = step_size)

    fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[10, 10], facecolor='black')

    idx = 0
    for img in slice_indices:
        axs.flat[idx].imshow(ndi.rotate(brain_vol_data[:, :, img], 90)[:, :, 0], cmap='gray')
        axs.flat[idx].axis('off')
        idx += 1
        
    plt.tight_layout()

    image = fig2img(fig)

    plt.close(fig) # Para que no muestre la imágen
    
    return image

In [9]:
# CreateGrid transform
class CreateGrid(object):
    """Creates a grid from the image
    """
    def __init__(self):
        True

    def __call__(self, sample):
        return transformGridImage(sample)

In [10]:
class RemoveTransparency(object):
    def __call__(self, sample):
      # La imagen se guarda con transparencia, removemos la dimension de indice 3
      return sample[0:3, :, :]

In [11]:
class ToLabelOutput(object):
    def __call__(self, label):
        if label == "CN":
            return 0
        elif label == "AD":
            return 1
        else:
            return 2 # MCI, LMCI, EMCI

In [12]:
class ADNIDataset(Dataset):
    """ADNI dataset."""

    def __init__(self, csv_file, root_dir, transform=None, target_transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
      return int(len(self.csv))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        studyID = self.csv.iloc[idx, 0]
        subjectID = self.csv.iloc[idx, 1]
        processFormat = self.csv.iloc[idx, 7]
        date = self.csv.iloc[idx, 9]
        diagnosis = self.csv.iloc[idx, 2]
        
        filename = None
        
        rglob = '*'+str(studyID)+'*.nii'
        samples = 0
        # for path in Path('ADNI-Full-PostProc').rglob(rglob):
        for path in Path(self.root_dir).rglob(rglob):
            filename = str(path)
            samples =+ 1
            
        if samples > 1:
            raise "Mas de un sample. Error"

        if not filename:
            raise Exception("Not found filename for index " + str(idx) + " y studyID " + studyID)

        # print("Leyendo: " + filename)
            
        brain_vol = nib.load(filename)

        image = brain_vol
        label = diagnosis
        
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [13]:
def printFile(text, file):
  print(text)
  file.write(text + "\n")

# Modelo

In [14]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=True):
    since = time.time()

    train_acc_history = []
    val_acc_history = []
    train_loss_history = []
    val_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        # No usamos el aux
                        #outputs, aux_outputs = model(inputs)
                        #loss1 = criterion(outputs, labels)
                        #loss2 = criterion(aux_outputs, labels)
                        #loss = loss1 + 0.4*loss2
                        outputs, aux_outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_loss_history, train_acc_history, train_loss_history

In [15]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Initialize and reshape inception

In [16]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained, 
                                       aux_logits = True)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        # num_ftrs = model_ft.AuxLogits.fc.in_features
        # model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        print("num featurs" + str(num_ftrs))
        # Fuente: https://github.com/bdrad/petdementiapub/blob/master/petdementia_source.py
        model_ft.fc = nn.Sequential(
          nn.Linear(num_ftrs,1024),
          nn.ReLU(),
          nn.Linear(1024,num_classes),
          # nn.Softmax() # innecesario, cross entrophy acepta logits
        )
          
        input_size = 512 

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=usePretrained)

# Print the model we just instantiated
# print(model_ft)

num featurs2048


In [17]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(input_size),
        #transforms.RandomHorizontalFlip(),
        CreateGrid(),
        transforms.ToTensor(),
        RemoveTransparency(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        CreateGrid(),
        transforms.ToTensor(),
        RemoveTransparency(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets

image_datasets = {
    'train': ADNIDataset(trainDatasetCSV, imagesFolder, transform = data_transforms['train'], target_transform =ToLabelOutput() ),
    'val': ADNIDataset(valDatasetCSV, imagesFolder, transform = data_transforms['val'], target_transform =ToLabelOutput() )
}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [18]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=learningRate)


Params to learn:
	 Conv2d_1a_3x3.conv.weight
	 Conv2d_1a_3x3.bn.weight
	 Conv2d_1a_3x3.bn.bias
	 Conv2d_2a_3x3.conv.weight
	 Conv2d_2a_3x3.bn.weight
	 Conv2d_2a_3x3.bn.bias
	 Conv2d_2b_3x3.conv.weight
	 Conv2d_2b_3x3.bn.weight
	 Conv2d_2b_3x3.bn.bias
	 Conv2d_3b_1x1.conv.weight
	 Conv2d_3b_1x1.bn.weight
	 Conv2d_3b_1x1.bn.bias
	 Conv2d_4a_3x3.conv.weight
	 Conv2d_4a_3x3.bn.weight
	 Conv2d_4a_3x3.bn.bias
	 Mixed_5b.branch1x1.conv.weight
	 Mixed_5b.branch1x1.bn.weight
	 Mixed_5b.branch1x1.bn.bias
	 Mixed_5b.branch5x5_1.conv.weight
	 Mixed_5b.branch5x5_1.bn.weight
	 Mixed_5b.branch5x5_1.bn.bias
	 Mixed_5b.branch5x5_2.conv.weight
	 Mixed_5b.branch5x5_2.bn.weight
	 Mixed_5b.branch5x5_2.bn.bias
	 Mixed_5b.branch3x3dbl_1.conv.weight
	 Mixed_5b.branch3x3dbl_1.bn.weight
	 Mixed_5b.branch3x3dbl_1.bn.bias
	 Mixed_5b.branch3x3dbl_2.conv.weight
	 Mixed_5b.branch3x3dbl_2.bn.weight
	 Mixed_5b.branch3x3dbl_2.bn.bias
	 Mixed_5b.branch3x3dbl_3.conv.weight
	 Mixed_5b.branch3x3dbl_3.bn.weight
	 Mixed_5b.b

In [19]:
# Setup the loss fxn
crossEntrophyWeigths = crossEntrophyWeigths.to(device)
criterion = nn.CrossEntropyLoss(crossEntrophyWeigths)

# Train and evaluate
model_ft, val_acc_hist, val_loss_hist, train_acc_hist, train_loss_hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/49
----------
train Loss: 1.1077 Acc: 0.3611
val Loss: 1.0512 Acc: 0.4833
Epoch 1/49
----------
train Loss: 0.9149 Acc: 0.6056
val Loss: 1.0896 Acc: 0.2833
Epoch 2/49
----------
train Loss: 0.6100 Acc: 0.7889
val Loss: 2.0438 Acc: 0.3667
Epoch 3/49
----------
train Loss: 0.3336 Acc: 0.8889
val Loss: 1.3360 Acc: 0.4833
Epoch 4/49
----------


KeyboardInterrupt: ignored

In [ ]:
torch.save(model_ft.state_dict(), os.path.join(experimentOutputFolder, experimentName + '.pth'))

## Validación

In [ ]:
lst = [ x.cpu().item() for x in val_acc_hist ]
plt.plot(lst)
ax = plt.gca()
plt.text(0.05, 0.9, 'FE = ' + str(feature_extract), transform=ax.transAxes)
plt.text(0.05, 0.8, 'LR = ' + str(learningRate), transform=ax.transAxes)
plt.text(0.05, 0.7, 'batch = ' + str(batch_size), transform=ax.transAxes)
plt.suptitle(experimentName + ' (acc set de validacion)')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.savefig(os.path.join(experimentOutputFolder, experimentName+'_val_acc.png'))
plt.show()

In [ ]:
# lst = [ x.cpu().item() for x in val_loss_hist ]
plt.plot(val_loss_hist)
ax = plt.gca()
plt.text(0.05, 0.3, 'FE = ' + str(feature_extract), transform=ax.transAxes)
plt.text(0.05, 0.2, 'LR = ' + str(learningRate), transform=ax.transAxes)
plt.text(0.05, 0.1, 'batch = ' + str(batch_size), transform=ax.transAxes)
plt.suptitle(experimentName + ' (loss set de validacion)')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.savefig(os.path.join(experimentOutputFolder, experimentName+'_val_loss.png'))
plt.show()

# Train

In [ ]:
lst = [ x.cpu().item() for x in train_acc_hist ]
ax = plt.gca()
plt.text(0.05, 0.9, 'FE = ' + str(feature_extract), transform=ax.transAxes)
plt.text(0.05, 0.8, 'LR = ' + str(learningRate), transform=ax.transAxes)
plt.text(0.05, 0.7, 'batch = ' + str(batch_size), transform=ax.transAxes)
plt.plot(lst)
plt.suptitle(experimentName + ' (accuracy set de train)')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.savefig(os.path.join(experimentOutputFolder, experimentName+'_train_acc.png'))
plt.show()

In [ ]:
ax = plt.gca()
plt.text(0.05, 0.3, 'FE = ' + str(feature_extract), transform=ax.transAxes)
plt.text(0.05, 0.2, 'LR = ' + str(learningRate), transform=ax.transAxes)
plt.text(0.05, 0.1, 'batch = ' + str(batch_size), transform=ax.transAxes)
plt.plot(train_loss_hist)
plt.suptitle(experimentName + ' (Loss set de train)')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.savefig(os.path.join(experimentOutputFolder, experimentName+'_train_loss.png'))
plt.show()

# Analisis de métricas

In [ ]:
def test_model(model,dataloaders,device):
    classStats = [{
        'fn': 0,
        'tn': 0,
        'tp': 0,
        'fp': 0,
        'n': 0,
    } for i in range(num_classes)]

    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Iteramos para chequear estadisticas
            for i, correctClass in enumerate(labels.data):
              predictedClass = int(preds[i].item())
              correctClass = int(correctClass.item())
              classStats[correctClass]['n'] += 1
              if correctClass == predictedClass:
                  classStats[correctClass]['tp'] += 1
                  for i in range(num_classes):
                      if i != correctClass:
                          classStats[correctClass]['tn'] += 1
              else:
                  classStats[correctClass]['fn'] += 1
                  classStats[predictedClass]['fp'] += 1
                  for i in range(num_classes):
                      if i != correctClass and i != predictedClass:
                          classStats[correctClass]['tn'] += 1
    return classStats

In [ ]:
stats = test_model(model_ft, dataloaders_dict, device)
print(stats)

In [ ]:
def printClassStats(stats):
  recall = sensitivity = stats['tp'] / (stats['tp'] + stats['fn']) # prob positive test result
  specificity = stats['tn'] / (stats['tn'] + stats['fp'])          # prob negative test result
  if stats['tp'] + stats['fp'] > 0:
    precision = stats['tp'] / (stats['tp'] + stats['fp'])            # prob of recognized positive actually correct
  else:
    precision = 1
    printFile("Setting precision as 1 but no positive value has been reported, so this is placeholder", f)
  f1 = 2 * (precision * recall) / ( precision + recall )
  printFile("Sensitivity (%): " + str(round(sensitivity * 100)), f)
  printFile("Specificity (%): " + str(round(specificity * 100)), f)
  printFile("Precision  (%): " + str(round(precision * 100)), f)
  printFile("F1 Score  (%): " + str(round(f1 * 100)), f)
  printFile("Number of images: " + str(stats['n']), f)
  return recall, specificity, precision, f1

In [ ]:
f = open(os.path.join(experimentOutputFolder, experimentName + "_stats.txt"), "w")
printFile("CN stats: ", f)
printClassStats(stats[0])
# AD
printFile("\nAD stats: ", f)
printClassStats(stats[1])
# MCI
printFile("\nMCI stats: ", f)
printClassStats(stats[2])
f.close()